In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import tensorflow as tf
data = pd.read_csv("fakenews_data.csv")
data['text'] = data['text'].values.astype('U')
# Split the data
X_train,X_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2, random_state=42)
#LSTM Model
# calculating maximum sequence length
def find_max_sequence_length(dataframe, column_name):
    # Assuming the specified column contains sequences (e.g., sentences)
    sequences = dataframe[column_name].tolist()

    # Calculate the maximum sequence length
    max_sequence_length = max(len(seq.split()) for seq in sequences)

    return max_sequence_length

max_length = find_max_sequence_length(data, 'text')
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 128, input_length=256),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2023-11-20 02:47:12.854121: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 02:47:12.906101: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 02:47:24.366118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-20 02:47:24.372383: I tensorflow

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
X_train1 = tokenizer.texts_to_sequences(X_train)
X_test1 = tokenizer.texts_to_sequences(X_test)
X_train2 = tf.keras.preprocessing.sequence.pad_sequences(X_train1, padding='post', maxlen=256)
X_test2 = tf.keras.preprocessing.sequence.pad_sequences(X_test1, padding='post', maxlen=256)

In [4]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

history = model.fit(
    X_train2, y_train,
    epochs=10,
    batch_size=30,
    validation_data=(X_test2, y_test),
    callbacks=[early_stop]
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-11-20 02:47:41.136343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2023-11-20 02:47:41.229102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-20 02:47:41.265298: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f96d994cd70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-20 02:47:41.265336: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GRID A100D-40C, Compute Capability 8.0
2023-11-20 02:47:41.270826: I tensorflow/compiler/mlir/tensorflow/utils/d

1198/1198 [==============================] - 77s 60ms/step - loss: 0.0544 - accuracy: 0.9828 - val_loss: 0.0102 - val_accuracy: 0.9969
Epoch 2/10
1198/1198 [==============================] - 47s 39ms/step - loss: 0.0100 - accuracy: 0.9974 - val_loss: 0.0072 - val_accuracy: 0.9984
Epoch 3/10
1198/1198 [==============================] - 47s 39ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 0.0075 - val_accuracy: 0.9988
Epoch 4/10
1198/1198 [==============================] - 45s 38ms/step - loss: 0.0050 - accuracy: 0.9986 - val_loss: 0.0081 - val_accuracy: 0.9984


In [10]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import tensorflow as tf
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def punctuation_removal(text):
    # Convert the NumPy array to a list of strings
    text_list = [str(item) for item in text]
    # Join the list into a string
    text_str = ' '.join(text_list)

    # Remove punctuation
    all_list = [char for char in text_str if char not in string.punctuation]
    clean_str = ''.join(all_list)
    
    return clean_str

def tokenize_text(input_text):
    # Remove punctuation
    clean_str = punctuation_removal(input_text)

    # Remove stopwords and tokenize
    stop = stopwords.words('english')
    tokens = word_tokenize(clean_str)
    tokens = [word for word in tokens if word.lower() not in stop]

    return tokens

#to calculate to emotion score
def calculate_emotion_score(tokens, eemotion_words):
    total_score = 0
    count = 0
    for word in tokens:
        if word in eemotion_words:
            total_score += eemotion_words[word]
            count += 1
    return total_score / max(1,count)

#code to classify emotions 
def classify_emotion(tokens, eemotion_words, word2vec_model):
    emotion_score = calculate_emotion_score(tokens, eemotion_words)
    
    # Use the 'wv' attribute for Word2Vec model
    token_vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]

    mean_vector = np.mean(token_vectors, axis=0)
    if emotion_score >= 0.8:
        return "Very Emotional Triggering"
    elif emotion_score >= 0.6:
        return "Emotionally Triggering"
    elif emotion_score >= 0.4:
        return "Neutral"
    elif emotion_score >= 0.2:
        return "Slightly Triggering"
    else:
        return "Safe Text"

# Define LSTM model
def create_lstm_model(input_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(10000, 128, input_length=input_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Function to preprocess input text
def preprocess_input(text):
    # Remove punctuation
    clean_str = punctuation_removal(text)

    # Remove stopwords
    stop = stopwords.words('english')
    clean_text = ' '.join([word for word in clean_str.split() if word not in stop])

    return clean_text

# Function to classify news and emotional intensity
def classify_news_and_emotion(input_text, lstm_model, emotion_words, word2vec_model):
    # Preprocess input text
    cleaned_text = preprocess_input(input_text)
    # Tokenize and pad the sequence
    input_sequence = tokenize_text(cleaned_text)
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], padding='post', maxlen=256)
    # Predict using LSTM model
    lstm_prediction = lstm_model.predict(padded_sequence)
    # Classify news as "fake news" or "real news"
    news_category = "fake news" if lstm_prediction >= 0.5 else "real news"
    # Tokenize the input text
    input_tokens = cleaned_text.split()
    # Calculate emotional intensity using fuzzy logic
    emotion_intensity = classify_emotion(input_tokens, emotion_words, word2vec_model)
    return news_category, emotion_intensity

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

negative_words = {
    'sad': 0.8, 'angry': 0.7, 'frustrated': 0.6, 'disappointed': 0.7,
    'upset': 0.8, 'miserable': 0.9, 'depressed': 0.8, 'dismal': 0.7,
    'unhappy': 0.6, 'troubled': 0.7, 'grim': 0.8, 'gloomy': 0.7,
    'distraught': 0.7, 'weary': 0.6, 'pessimistic': 0.7, 'annoyed': 0.6,
    'irritated': 0.7, 'hostile': 0.8, 'bitter': 0.7, 'resentful': 0.6,
    'discouraged': 0.7, 'hopeless': 0.8, 'dejected': 0.7, 'sorrowful': 0.6,
    'overwhelmed': 0.8, 'exhausted': 0.7, 'numb': 0.6, 'dreadful': 0.8,
    'unpleasant': 0.7, 'apprehensive': 0.6, 'repulsive': 0.7, 'hateful': 0.8,
    'detestable': 0.7, 'abominable': 0.6, 'abhorrent': 0.7, 'repugnant': 0.8,
    'disgusted': 0.7, 'revolting': 0.6, 'disheartened': 0.7, 'discontented': 0.8,
    'displeased': 0.7, 'disgruntled': 0.6, 'mournful': 0.7, 'wretched': 0.8,
    'desolate': 0.7, 'forlorn': 0.6, 'grief-stricken': 0.7, 'anguished': 0.8,
    'desperate': 0.7, 'woeful': 0.6, 'woebegone': 0.7, 'melancholy': 0.8,
    'pitiful': 0.7, 'pathetic': 0.6, 'heartrending': 0.7, 'agonizing': 0.8,
    'worn-out': 0.6, 'fatigued': 0.7, 'drained': 0.6, 'exasperated': 0.7,
    'irksome': 0.6, 'enraged': 0.8, 'infuriated': 0.7, 'livid': 0.8,
    'indignant': 0.7, 'outraged': 0.8, 'incensed': 0.7, 'riled': 0.6,
    'troubled': 0.7, 'dismayed': 0.8, 'crestfallen': 0.7, 'disillusioned': 0.6,
    'disgruntled': 0.7, 'displeased': 0.6, 'irritable': 0.7, 'querulous': 0.6,
    'cross': 0.7, 'discontented': 0.6, 'disgusted': 0.7, 'revolted': 0.6,
    'abhorrent': 0.7, 'repellent': 0.6, 'disastrous': 0.8, 'woeful': 0.7,
    'mournful': 0.8, 'tragic': 0.7, 'grievous': 0.8, 'melancholic': 0.7,
    'heartbroken': 0.8, 'crestfallen': 0.7, 'downcast': 0.6, 'unfortunate': 0.7,
    'unlucky': 0.6, 'unhappy': 0.7, 'forlorn': 0.6, 'gloomy': 0.7,
    'dreary': 0.6, 'bleak': 0.7, 'desperate': 0.6, 'hopeless': 0.7,
    'dismal': 0.6, 'pessimistic': 0.7, 'fatalistic': 0.6, 'unfavorable': 0.7,
    'adverse': 0.6, 'hostile': 0.7, 'antagonistic': 0.6, 'inhospitable': 0.7,
    'unsympathetic': 0.6, 'callous': 0.7, 'indifferent': 0.6, 'apathetic': 0.7,
    'listless': 0.6, 'uninterested': 0.7, 'bored': 0.6, 'weary': 0.7,
    'tired': 0.6, 'exhausted': 0.7, 'fatigued': 0.6, 'drained': 0.7,
    'weary': 0.6, 'burned-out': 0.7, 'worn-out': 0.6, 'lethargic': 0.7,
    'sluggish': 0.6, 'lifeless': 0.7, 'apathetic': 0.6, 'indifferent': 0.7,
    'unresponsive': 0.6, 'insensitive': 0.7, 'cold': 0.6, 'aloof': 0.7,
    'detached': 0.6, 'unemotional': 0.7, 'stoic': 0.6, 'expressionless': 0.7,
    'robotic': 0.6, 'mechanical': 0.7
}

trigger_words = {
    'death': 0.9, 'shot' : 0.9,'failure': 0.8, 'accident': 0.7, 'tragedy': 0.8,
    'disaster': 0.9, 'crisis': 0.8, 'heartbreak': 0.7, 'calamity': 0.8,
    'horrific': 0.9, 'devastating': 0.8, 'trauma': 0.7, 'nightmare': 0.8,
    'catastrophe': 0.8, 'calamitous': 0.7, 'cataclysmic': 0.8, 'apocalyptic': 0.7,
    'devastation': 0.8, 'ruin': 0.7, 'carnage': 0.8, 'destruction': 0.7,
    'misfortune': 0.8, 'mishap': 0.7, 'catastrophic': 0.8, 'accidental': 0.7,
    'fatal': 0.8, 'cruel': 0.7, 'harmful': 0.8, 'perilous': 0.7,
    'threatening': 0.8, 'menacing': 0.7, 'ominous': 0.8, 'foreboding': 0.7,
    'dreadful': 0.8, 'fateful': 0.7, 'unfortunate': 0.8, 'troublesome': 0.7,
    'dire': 0.8, 'unlucky': 0.7, 'unfavorable': 0.8, 'unfortunate': 0.7,
    'hazardous': 0.8, 'dangerous': 0.7, 'threatening': 0.8, 'ominous': 0.7,
    'horrifying': 0.8, 'terrifying': 0.7, 'shocking': 0.8, 'appalling': 0.7,
    'dreadful': 0.8, 'ghastly': 0.7, 'spine-chilling': 0.8, 'hair-raising': 0.7,
    'catastrophe': 0.8, 'calamitous': 0.7, 'cataclysmic': 0.8, 'apocalyptic': 0.7,
    'devastation': 0.8, 'ruin': 0.7, 'carnage': 0.8, 'destruction': 0.7,'terrorist' : 0.8, 
    'misfortune': 0.8, 'mishap': 0.7, 'catastrophic': 0.8, 'accidental': 0.7,
    'fatal': 0.8, 'cruel': 0.7, 'harmful': 0.8, 'perilous': 0.7,
    'threatening': 0.8, 'menacing': 0.7, 'ominous': 0.8, 'foreboding': 0.7,
    'dreadful': 0.8, 'fateful': 0.7, 'unfortunate': 0.8, 'troublesome': 0.7,
    'dire': 0.8, 'unlucky': 0.7, 'unfavorable': 0.8, 'unfortunate': 0.7,
    'hazardous': 0.8, 'dangerous': 0.7, 'threatening': 0.8, 'ominous': 0.7,
    'horrifying': 0.8, 'terrifying': 0.7, 'shocking': 0.8, 'appalling': 0.7,
    'dreadful': 0.8, 'ghastly': 0.7, 'spine-chilling': 0.8, 'hair-raising': 0.7,
    'abysmal': 0.8, 'bleak': 0.7, 'ghastly': 0.8, 'grim': 0.7,
    'heartbreaking': 0.8, 'unbearable': 0.7, 'gut-wrenching': 0.8, 'disconcerting': 0.7,
    'perturbing': 0.8, 'unsettling': 0.7, 'alarming': 0.8, 'disquieting': 0.7,
    'unnerving': 0.8, 'foreboding': 0.7, 'threatening': 0.8, 'menacing': 0.7,
    'hostile': 0.8, 'noxious': 0.7, 'pernicious': 0.8, 'destructive': 0.7,
    'debilitating': 0.8, 'ruinous': 0.7, 'catastrophically': 0.8, 'tragic': 0.7,
    'devastating': 0.8, 'ruinous': 0.7
}

df = pd.read_csv("fakenews_data.csv")
# Convert 'text' column to strings, handling potential NaN values
df['text'] = df['text'].astype(str)
sentences = df['text'].values

# Create and train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)
# Save the trained Word2Vec model (optional)
word2vec_model.save("word2vec_model.model")

In [ ]:
# Example usage
input_tweet = input("Enter your tweet: ")
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts([input_tweet])

lstm_model = model

# Tokenize the input text
sequences = tokenizer.texts_to_matrix([input_tweet], mode='count')

# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=256, padding='post')

# Convert the padded_sequences to a list
padded_sequences_list = [padded_sequences[0]]
emotion_words = {**negative_words, **trigger_words}

# Call the function to classify news and emotional intensity
news_category, emotion_intensity = classify_news_and_emotion(padded_sequences_list, lstm_model, emotion_words, word2vec_model)

# Print the results
print("News Category:", news_category)
print("Emotion Intensity:", emotion_intensity)